<a href="https://colab.research.google.com/github/sonosergi/FitCoach/blob/main/Workout_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### IMPORT MODULES


In [ ]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import zipfile
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
from datetime import datetime
from collections import deque
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import InceptionResNetV2

### IMPORT DATA

In [ ]:
url = '/content/drive/MyDrive/Colab Notebooks/iTrainer/archive.zip'
zip_ref = zipfile.ZipFile(url, "r")
zip_ref.extractall("/tmp/Data-Keggle")
zip_ref.close()

### HYPERPARAMETER

In [ ]:
height = 256
width = 256
channels = 3
BATCH_SIZE = 32
SEED = 127
img_shape = (height, width, channels)
IMG_SIZE = (height, width)
CLASSES = 22

### ETL DATA

In [ ]:
DATA_DIR = '/tmp/Data-Keggle'

# dividir el conjunto de datos en entrenamiento (80%) y prueba (20%)
training_data = tf.keras.utils.image_dataset_from_directory(DATA_DIR,
  labels = 'inferred',
  label_mode = 'categorical',
  validation_split = 0.3,
  subset = 'training',
  image_size = IMG_SIZE,
  shuffle = True,
  batch_size = BATCH_SIZE,
  seed = 127
  )

test_data = tf.keras.utils.image_dataset_from_directory(DATA_DIR,
  labels = 'inferred',
  label_mode = 'categorical',
  validation_split = 0.3,
  subset = 'validation',
  image_size = IMG_SIZE,
  shuffle = True,
  batch_size = BATCH_SIZE,
  seed = 127
)

labels = training_data.class_names
print(labels)

with open('workout_label.txt', 'w') as f:
    for workout_class in labels:
        f.write(f'{workout_class}\n')

In [ ]:
#training_data = ImageDataGenerator(rescale = 1/255)
#test_data = ImageDataGenerator(rescale = 1/255)

### DATA AUGMENTATION

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2),
  tf.keras.layers.RandomContrast(0.2),
  tf.keras.layers.GaussianNoise(0.2),
  tf.keras.layers.RandomBrightness(0.2)
])

training_data = training_data.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
def show_img(data):
    plt.figure(figsize=(10,10))
    for images, labels in data.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            ax.imshow(images[i].numpy().astype("uint8"))
            ax.axis("off")

In [ ]:
show_img(training_data)

### MODEL

In [ ]:
pre_trained_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=img_shape, pooling='avg')

for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

#### Congelamos el modelo 

In [ ]:
x = pre_trained_model.output
x = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(len(labels), activation='softmax')(x)

iTrainer = tf.keras.models.Model(inputs = pre_trained_model.input, 
                                      outputs = predictions
                                     )

iTrainer.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy']
                     )

iTrainer.summary()

### TRAIN

In [ ]:
early_stopping_callback = EarlyStopping(monitor = 'loss', 
                                        patience = 5, 
                                        mode = 'auto', 
                                        restore_best_weights = True
                                       )

history = iTrainer.fit(training_data,
                            validation_data = test_data,
                            epochs = 200,
                            callbacks = [early_stopping_callback]
                           )

In [ ]:
current_datetime = datetime.now()

# Save Model
iTrainer.save(f'iTrainer_{current_datetime}')

# Save .h5 model
iTrainer.save(f'iTrainer_{current_datetime}.h5')

# Convert the model to tflite
converter = tf.lite.TFLiteConverter.from_saved_model(f'/content/drive/MyDrive/Colab Notebooks/iTrainer/Models/iTrainer_{current_datetime}')
#converter = tf.lite.TFLiteConverter.from_saved_model(f'./iTrainer_{current_datetime}')
tflite_model = converter.convert()

# Save the tflite model
with open(f'iTrainer{current_datetime}.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter --input_format keras perros-gatos-cnn-ad.h5 carpeta_salida